In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
# 모듈 삽입

import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt 
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler


In [0]:
# 드라이브에 저장된 데이터 불러오기

df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/project_nuclear/project_processed_data/output/v2_train_data.csv")


In [5]:
df.head

<bound method NDFrame.head of            V0000     V0001     V0002  ...  V5119  V5120  label
0      30.474394  8.691177  8.714483  ...   85.4    0.0    110
1      30.470463  8.736521  8.682769  ...   85.4    0.0    110
2      30.465427  8.753559  8.663426  ...   85.4    0.0    110
3      30.458532  8.715056  8.714854  ...   85.4    0.0    110
4      30.475773  8.790241  8.735125  ...   85.4    0.0    110
...          ...       ...       ...  ...    ...    ...    ...
41395  30.476523  8.680374  8.689265  ...   85.4    0.0    156
41396  30.464995  8.818355  8.688116  ...   85.4    0.0    156
41397  30.483490  8.737560  8.706527  ...   85.4    0.0    156
41398  30.459609  8.696491  8.685300  ...   85.4    0.0    156
41399  30.463722  8.707559  8.710813  ...   85.4    0.0    156

[41400 rows x 5122 columns]>

In [6]:
# 전체 train data

train_x = df.drop('label', axis=1, inplace=False)
train_y = df['label']
train_y_h = pd.get_dummies(train_y).values

print(train_x.shape)
print(train_y.shape)
print(train_y_h.shape)

train_y_h = pd.DataFrame(train_y_h)

(41400, 5121)
(41400,)
(41400, 198)


In [7]:
########################
# 학습용 x 데이터 정규화 : train_x_n
########################

scaler=MinMaxScaler()
train_x_n = scaler.fit_transform(train_x)
train_x_n = pd.DataFrame(train_x_n)
train_x_n.shape

(41400, 5121)

In [0]:
'''########################
# 학습용 x 데이터 범주화 : train_x_p
########################
train_x_p = train_x

for i in range(1,train_x_p[1]):
    q1,q2,q3 = np.percentile(train_x.iloc[:,i],[25,50,75])
    train_x.loc[train_x.iloc[:,i] <= q1, train_x.columns[i]] = 0
    train_x.loc[(train_x.iloc[:,i] > q1) & (train_x.iloc[:,i] <= q2), train_x.columns[i]] = 1
    train_x.loc[(train_x.iloc[:,i]> q2) & (train_x.iloc[:,i] <= q3), train_x.columns[i]] = 2
    train_x.loc[train_x.iloc[:,i] > q3, train_x.columns[i]] = 3'''

#########################################################

In [8]:
tf.reset_default_graph()

X = tf.placeholder(shape=[None, train_x_n.shape[1]], dtype=tf.float32)
Y = tf.placeholder(shape=[None, 198], dtype=tf.float32)
dout_rate = tf.placeholder(dtype=tf.float32)

W1 = tf.get_variable("weight1", shape=[train_x_n.shape[1],500], initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal(shape=[500]), name='bias1')

# dropout 사용!!!
_layer1 = tf.nn.relu(tf.matmul(X,W1) + b1)
# layer1 = tf.nn.dropout(_layer1, rate=0.3)
# 다만, 이를 일일이 적지 않고 새로운 placeholder로 준다.
layer1 = tf.nn.dropout(_layer1, rate=dout_rate)

W2 = tf.get_variable("weight2", shape=[500,500], initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal(shape=[500]), name='bias2')
_layer2 = tf.nn.relu(tf.matmul(layer1,W2) + b2)
layer2 = tf.nn.dropout(_layer2, rate=dout_rate)

W3 = tf.get_variable("weight3", shape=[500,198], initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal(shape=[198]), name='bias3')

logit = tf.matmul(layer2, W3) + b3
H = tf.nn.relu(logit)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logit, labels=Y))

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [0]:
# AdamOptimizer 사용
train = tf.train.AdamOptimizer(learning_rate=0.05).minimize(cost)

sess = tf.Session()
sess.run(tf.global_variables_initializer())



for step in range(1000):
    _, cost_val = sess.run([train, cost], feed_dict={X: train_x_n,
                                                     Y: train_y_h,
                                                     dout_rate: 0})
    if step % 10 == 0:
        print(f'cost값은:{cost_val}')
        


'''

num_of_epoch = 50
batch_size = 100

for step in range(num_of_epoch):    
    num_of_iter = int(train_x.shape[0] / batch_size)
    cost_val = 0
    
    for i in range(num_of_iter):
        batch_x = train_x[i*batch_size:(i+1)*batch_size]
        batch_y = train_y[i*batch_size:(i+1)*batch_size]
        _, cost_val = sess.run([train, cost], feed_dict={X: batch_x,
                                                         Y: batch_y,
                                                         dout_rate: 0})
    if step % 3 == 0:
        print(f'cost: {cost_val}')
'''

In [0]:
'''predict = tf.argmax(H,1)
correct = tf.equal(predict, tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct, dtype=tf.float32))
print(f'정확도:{sess.run(accuracy, feed_dict = {X: test_x_data, Y: test_y_data})}')'''

In [0]:
# 예측용 x data 불러오기

test = pd.read_csv("/content/drive/My Drive/Colab Notebooks/project_nuclear/project_processed_data/output/test_id_1.csv")

In [14]:
# 예측용 x data
test.to_csv("v2_test_data", index=)

,Unnamed: 0,V0000,V0001,V0002,V0003,V0004,V0005,V0006,V0007,V0008,V0009,V0010,V0011,V0012,V0013,V0014,V0015,V0016,V0017,V0018,V0019,V0020,V0021,V0022,V0023,V0024,V0025,V0026,V0027,V0028,V0029,V0030,V0031,V0032,V0033,V0034,V0035,V0036,V0037,V0038,...,V5081,V5082,V5083,V5084,V5085,V5086,V5087,V5088,V5089,V5090,V5091,V5092,V5093,V5094,V5095,V5096,V5097,V5098,V5099,V5100,V5101,V5102,V5103,V5104,V5105,V5106,V5107,V5108,V5109,V5110,V5111,V5112,V5113,V5114,V5115,V5116,V5117,V5118,V5119,V5120
0,1000,30.465741,8.618514,8.705075,8.730912,8.699214,181.327530,201.889419,2.393806e-19,0.0,0.003496,0.0,4.554062e-19,-7.382478e-18,274.136413,273.328885,274.551345,271.311643,36.019012,35.946994,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,18.345175,2.572785,2.566243,36.022586,35.988482,36.007483,35.993519,35.998178,35.998873,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,...,1.0,1.0,1.0,110.940222,110.849652,-0.294763,-0.229694,-0.172642,-0.181698,43.203640,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,60.0,0.0,0.0,0.000004,85.4,0.0
1,1000,30.477302,8.642689,8.713423,8.732450,8.694666,203.347675,155.790045,-1.808861e-19,0.0,0.001969,0.0,-1.494040e-18,4.637735e-18,276.134833,272.942135,274.933115,271.659800,36.032406,35.873491,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,18.339668,2.573976,2.567887,35.977842,36.008113,36.012230,35.994503,36.000896,36.014200,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,...,1.0,1.0,1.0,110.767512,110.853183,-0.286671,-0.232155,-0.177659,-0.164828,43.195677,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,60.0,0.0,0.0,0.000005,85.4,0.0
2,1000,30.478336,8.675928,8.729837,8.672877,8.710215,196.673652,227.039249,6.236627e-19,0.0,-0.002462,0.0,1.136759e-19,-6.393514e-18,274.930430,273.067079,273.934830,275.266410,36.034085,35.977848,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,18.335099,2.574067,2.567056,35.999382,35.992636,35.993587,36.017950,36.009234,35.999601,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,...,1.0,1.0,1.0,110.831842,110.935070,-0.299707,-0.249112,-0.193733,-0.170861,43.192395,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,60.0,0.0,0.0,0.000016,85.4,0.0
3,1000,30.462904,8.733765,8.706455,8.691974,8.696285,194.365551,167.436935,2.845012e-20,0.0,-0.000045,0.0,-3.321358e-18,-2.742367e-18,273.166307,273.397679,274.132773,270.916462,36.054774,36.018231,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,18.350084,2.573044,2.564821,35.996194,35.999353,35.996708,35.996904,35.995837,36.002971,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,...,1.0,1.0,1.0,110.936369,110.873987,-0.284462,-0.240052,-0.166091,-0.164244,43.195476,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,60.0,0.0,0.0,-0.000010,85.4,0.0
4,1000,30.483675,8.807382,8.680733,8.713651,8.664766,205.369347,154.245975,-2.085638e-19,0.0,-0.000713,0.0,6.620339e-19,5.115456e-18,275.621504,275.613200,274.764093,272.904413,35.973796,36.008455,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,18.358404,2.574133,2.566180,36.035024,35.991830,36.009338,36.006742,36.005264,36.001450,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,...,1.0,1.0,1.0,110.987712,110.927734,-0.283827,-0.230030,-0.228277,-0.139462,43.193896,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,60.0,0.0,0.0,-0.000008,85.4,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35995,999,30.462765,8.636107,8.717540,8.714714,8.673677,209.395256,221.627222,-5.455288e-19,0.0,-0.000699,0.0,1.039294e-18,-6.110042e-18,274.925221,274.601394,274.368920,273.272183,36.016504,36.030044,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001

In [0]:
scaler=MinMaxScaler()
test_x = scaler.fit_transform(test_x)
test_x = df.drop('Unnamed: 0', axis=1, inplace=False)
test_x = pd.DataFrame(test_x)
test_x.shape

(41400, 5122)

In [0]:
pre = sess.run(tf.argmax(H,1),
                  feed_dict={X: test_x,
                             dout_rate: 0})
print(pre)

[23 23 23 ... 23 23 23]


In [0]:
submission = pd.DataFrame(data=pred)
submission.index = test.index
submission.index.name = 'id'
submission = submission.sort_index()
submission = submission.groupby('id').mean()
submission.to_csv('submission_v2.csv', index=True)

In [0]:
'''predict = tf.argmax(H,1)
correct = tf.equal(predict, tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct, dtype=tf.float32))

print(f'정확도:{sess.run(accuracy, feed_dict={X:mnist.test.images, Y: mnist.test.labels,dout_rate: 0})}')'''

#########################################################

In [0]:
X_train = train.drop(['label'], axis=1)
y_train = train['label']
model = RandomForestClassifier(random_state=0, verbose=1, n_jobs=-1)
model.fit(X_train, y_train)

In [0]:
# model = joblib.load('model.pkl') 
pred = model.predict_proba(test)